In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

# 1. 데이터 불러오기
df = pd.read_csv('../data/secom.csv')

print(f"샘플: {df.shape[0]:,}개")
print(f"특성: {df.shape[1]:,}개")

# 타겟 확인
target_col = 'Pass/Fail'
print(f"\n타겟 분포:")
print(df[target_col].value_counts())

샘플: 1,567개
특성: 592개

타겟 분포:
Pass/Fail
-1    1463
 1     104
Name: count, dtype: int64


In [2]:
# 2. 타겟 변환 및 특성 분리

# 타겟 변수 변환
Y = df[target_col].map({-1: 0, 1: 1})  # -1: 정상 → 0, 1: 불량 → 1

# 특성 변수 분리
X = df.drop(columns=[target_col])

# Time 컬럼 datetime 변환 및 Feature Engineering
X['Time'] = pd.to_datetime(X['Time'])

# 시간 관련 특성 생성
X['hour'] = X['Time'].dt.hour  # 시간대 (0-23)
X['dayofweek'] = X['Time'].dt.dayofweek  # 요일 (0=월요일)
X['day'] = X['Time'].dt.day  # 날짜
X['time_gap'] = X['Time'].diff().dt.total_seconds().fillna(0)  # 측정 간격(초)

# Time 원본 컬럼 제거 (datetime은 모델에 못 넣음)
X = X.drop(columns=['Time'])

# 숫자형만 선택
X = X.select_dtypes(include=[np.number])

print(f"\n특성 개수: {X.shape[1]}개")


최종 특성 개수: 594개

특성 개수: 594개


In [3]:
# 3. 결측치 처리

# 결측치 확인
missing_before = X.isnull().sum().sum()
missing_pct = missing_before / (X.shape[0] * X.shape[1]) * 100
print(f"\n결측치 개수 (처리 전): {missing_before}")
print(f"결측치 비율 (처리 전): {missing_pct:.2f}%")

# 중앙값으로 결측치 대체
X_filled = X.fillna(X.median())

# 결측치 확인 (처리 후)
missing_after = X_filled.isnull().sum().sum()
print(f"\n결측치 개수 (처리 후): {missing_after}")


결측치 개수 (처리 전): 41951
결측치 비율 (처리 전): 4.51%

결측치 개수 (처리 후): 0


In [4]:
# 4. 상수 특성 제거

# 상수 특성 확인
constant_features = []
for col in X_filled.columns:
    if X_filled[col].nunique() <= 1:
        constant_features.append(col)
print(f"\n상수 특성 개수: {len(constant_features)}개")

# 상수 특성 제거
X_cleaned = X_filled.drop(columns=constant_features)
print(f"상수 특성 제거 후 특성 개수: {X_cleaned.shape[1]}개")


상수 특성 개수: 116개
상수 특성 제거 후 특성 개수: 478개


In [5]:
# 5. 스케일링

# 스케일링 전 데이터 확인
print(f'평균: {X_cleaned.mean()[:5].values}')
print(f'표준편차: {X_cleaned.std()[:5].values}')

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cleaned)

# df로 변환
X_scaled = pd.DataFrame(X_scaled, columns=X_cleaned.columns)

# 스케일링 후 데이터 확인
print(f'\n스케일링 후 평균: {X_scaled.mean()[:5].values}')
print(f'스케일링 후 표준편차: {X_scaled.std()[:5].values}')

평균: [3014.44155073 2495.8661104  2200.55195801 1395.38347409    4.17128054]
표준편차: [ 73.48084148  80.22814345  29.38097315 439.83732987  56.10372105]

스케일링 후 평균: [ 1.62332035e-15 -5.07372631e-15 -5.71336214e-15 -1.26963603e-16
  0.00000000e+00]
스케일링 후 표준편차: [1.00031923 1.00031923 1.00031923 1.00031923 1.00031923]


In [6]:
# 6. 데이터 분할 (Train / Test)

# 80:20 비율로 분할. stratifiy 옵션으로 클래스 비율 유지
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, Y, test_size=0.2, random_state=42, stratify=Y
)

# train
print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {y_train.shape}')
print(f'정상: {(y_train == 0).sum()}개, 불량: {(y_train == 1).sum()}개')
print(f'불량 비율: {(y_train == 1).mean() * 100:.2f}%')

# test
print(f'\nX_test shape: {X_test.shape}')
print(f'Y_test shape: {y_test.shape}')
print(f'정상: {(y_test == 0).sum()}개, 불량: {(y_test == 1).sum()}개')
print(f'불량 비율: {(y_test == 1).mean() * 100:.2f}%')

X_train shape: (1253, 478)
Y_train shape: (1253,)
정상: 1170개, 불량: 83개
불량 비율: 6.62%

X_test shape: (314, 478)
Y_test shape: (314,)
정상: 293개, 불량: 21개
불량 비율: 6.69%


In [9]:
# 7. 데이터 저장

# 디렉토리 생성
if not os.path.exists('../processed_data'):
    os.makedirs('../processed_data')
    print("'processed_data' 디렉토리 생성")

# 데이터 저장(csv)
X_train.to_csv('../processed_data/X_train.csv', index=False)
X_test.to_csv('../processed_data/X_test.csv', index=False)
y_train.to_csv('../processed_data/y_train.csv', index=False, header=['Pass/Fail'])
y_test.to_csv('../processed_data/y_test.csv', index=False, header=['Pass/Fail'])
print("\n데이터 저장 완료")
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

# 스케일러 저장(pickle)
with open('../processed_data/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("스케일러 저장 완료")




데이터 저장 완료
X_train shape: (1253, 478)
X_test shape: (314, 478)
y_train shape: (1253,)
y_test shape: (314,)
스케일러 저장 완료


# 전처리 완료 요약

## 입력
- secom.csv (1567 × 592)

## 처리
1. 결측치 4.52% → 중앙값 대체
2. 상수 특성 116개 제거
3. StandardScaler 적용
4. Train/Test 분리 (80:20, stratify)

## 출력
- Train: 1,253개 (불량 6.62%)
- Test: 314개 (불량 6.69%)
- 최종 특성: 474개